In [1]:
from __future__ import print_function, division
import os
import sys
import imp
import numpy as np

import mat_neuron._model as mat
from dstrf import strf, mle, io

# plotting packages
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package

In [15]:
import yaml
test = """
a:
    b: 1
    c: 2
"""

config = yaml.load(test)

In [ ]:
def load_rothman(model,nspec,t_dsample,compress=1,gammatone=True,root="/scratch/mcb2x/sigma5/"):
    stimfiles = []
    spikefiles = []
    for f in next(walk(root+model))[2]:
        split = f.split(".")
        if len(split)==2:
            if split[1] =="wav":
                stimfiles.append(f)
        else:
            if f[:5] == "spike":
                spikefiles.append(f)
    spikefiles = np.sort(spikefiles)
    stimfiles = np.sort(stimfiles)

    stims,durations = load_sound_data(stimfiles,root+model,dsample=t_dsample,sres=nspec,gammatone=gammatone,compress=compress)
    spikes_data, spiky_data = load_spikes_data(spikefiles,root+model, durations,ts_file=False)
    
    return stims,durations,spikes_data,spiky_data

In [5]:
# model parameters: (ω, α1, α2, τ1, τ2, tref)
matparams = np.asarray([7, 100, 2, 10, 200, 2], dtype='d')
model_dt = 0.5
nlin = "exp"

matparams_i = [0,1,2]
matparams_n = len(matparams_i)
matparams_f = matparams[matparams_i]

# STRF: keep this very simple for proof of principle
stim_dt = 3.0
upsample = int(stim_dt / model_dt)
spec_window = 4.0
spec_compress = 10
f_min = 1.0
f_max = 8.0
nfreq = 20
ntau  = 40
# these parameters influence how the STRF dimensional reduction will work
ntbas = 10       # number of temporal basis functions
krank = 2        # rank of bilinear approximation
kcosbas = strf.cosbasis(ntau, ntbas)
ntbas = kcosbas.shape[1]

In [193]:
def load_sound_data(files,root="",windowtime=256,ovlerlap=10,f_min=500,f_max=8000,gammatone=False,
                    dsample=10,sres=15,compress=0):
    stims = []
    durations = []

    for f in files:
        Fs, wave = read(root+f)
        wt = windowtime/Fs
        ovl = ovlerlap/Fs

        duration = int(1000*len(wave)/Fs)
        durations.append(duration)
        if gammatone:
            Pxx = gg.gtgram(wave,Fs,wt,ovl,sres,f_min,f_max)
            Pxx = 10*np.log10(Pxx+compress)

        else:
            w = np.hanning(int(windowtime))
            Pxx = libtfr.stft(wave, w, int(w.size * .1))
            freqs, ind = libtfr.fgrid(Fs, w.size, [f_min, f_max])
            Pxx = Pxx[ind,:]
            Pxx = 10*np.log10(Pxx+compress)
            Pxx = resample(Pxx,sres)
        Pxx = resample(Pxx,int(duration/dsample),axis=1)
        stims.append(Pxx)
    return stims,durations

def load_spikes_data(files,root="",durations=[],ts_file=True,delim=" "):
    spikes_data = []
    spiky_data = []
    for d,f in enumerate(files):
        # load spike data
        dur = durations[d]
        if ts_file: stim_spikes = [filter(lambda x: x >= 0 and x <= dur, s) for s in ts.read(open(root + f))[0]]
        else: stim_spikes = [filter(lambda x: x >= 0 and x <= dur, map(float,s.split(delim))) for s in open(root + f)]
        spikes_data.append(stim_spikes)
        stim_spiky = [spk.SpikeTrain(s,[0,durations[d]]) for s in stim_spikes]
        spiky_data.append(stim_spiky)
        
    return spikes_data, spiky_data

def load_rothman(cell, root, window, step, **specargs):
    """Load stimulus and response data from SNR4synhg

    Additional keyword arguments are passed to load_stimulus()
    """
    spikesroot = os.path.join(root, cell)
    stimroot = os.path.join(root, "stims")

    out = []
    for fname in glob.iglob(os.path.join(root,cell,"spike*")):
        stim = "stim{}-0.wav".format(fname[-2:])
        spec, dur = io.load_stimulus(os.path.join(stimroot,stim), window, step, **specargs)
        spikes = []
        for f in open(fname):
            l = f.split(" ")
            if l[0] == "\n": spikes.append(np.empty([0],dtype=float))
            else: 
                spikes.append(np.asarray(l,dtype=float))
        spikes = tuple(spikes)
        out.append({"cell_name": cell,
                    "stim_name": stim,
                    "duration": dur,
                    "stim": spec,
                    "stim_dt": step,
                    "spikes": spikes})
    return out

In [194]:
import glob

cell = "b-tonic-4"
root = "/scratch/mcb2x/modeldata/SNR4synhg/"

out= load_rothman(cell,root,spec_window,stim_dt)